In [63]:
from typing import Tuple
from collections import defaultdict
import numpy as np
import pandas as pd


class Trace:

    def __init__(self, filename: str, benchmarks=None):
        self.baseline = set(benchmarks) if benchmarks else set()
        self.scores, self.expressions, generations_by_task = parse_log(filename, baseline=self.baseline)
        self.comparison, self.d_scores = comparisons(self.scores)
        self.in_comparison, self.in_d_scores = comparisons(self.scores, sample="in-sample")
        self.generations_by_task = pd.Series(generations_by_task, name="generations")

    @property
    def most_frequent_solutions_by_length(self):
        for length, expressions in sorted(self.expressions.items()):
            m = max(set(expressions), key=expressions.count)
            yield m, expressions.count(m)
            # print(f" Found {len(expressions):3d} expressions of length {length}. Most frequent: {m} ({expressions.count(m)} times)")


def parse_log(file, with_prefix=False, baseline=None):
    with open(file) as fh:
        lines = fh.readlines()

    p = 'INFO:root:' if with_prefix else ''

    definitions = [line for line in lines if ':=' in line]
    baseline = baseline if baseline else set()

    print("The predefined defaults are:")
    for line in definitions:
        if ':=' in line:
            print(f" * {line[len(p):-1]}")
            baseline.add(line[len(p):].split(' :=')[0])

    task_starts = [i for i, line in enumerate(lines) if "START_TASK:" in line]
    in_sample_starts = [i for i, line in enumerate(lines) if "Evaluating in sample:" in line]
    out_sample_starts = [i for i, line in enumerate(lines) if "Evaluating out-of-sample:" in line]

    def parse_evaluation_line(line) -> Tuple[str, int, float]:
        """ Parse an evaluation line, returning the expression or name, its 'length' and the score.

        e.g. INFO:root:[make_tuple(p, mkd)|0.8893]\n -> 'make_tuple(p, mkd)', 1, 0.8893
        Length is 0 for benchmark problems.
        """
        start, pipe, end = line.find('['), line.find('|'), line.find(']')
        expression = line[start + 1: pipe]
        if ':' in expression:  # For the baseline expressions, record them by name
            expression = expression[:expression.find(':')]
        expression_length = expression.count('(')
        return expression, expression_length, float(line[pipe + 1: end])

    tasks = [int(line[:-1].split(": ")[-1]) for line in lines if "START_TASK:" in line]
    idx = pd.MultiIndex.from_product([tasks, ["in-sample", "out-sample"]], names=['task', 'sample-type'])
    df = pd.DataFrame(index=idx, columns=["length-1", "length-2", "length-3", "final", *baseline], dtype=float)

    expressions_by_length = defaultdict(list)
    generations_by_task = {}

    for task_start, in_start, out_start, next_task in zip(task_starts, in_sample_starts,
                                                          out_sample_starts,
                                                          task_starts[1:] + [-len(baseline)*2]):
        # start line looks like: INFO:root:START_TASK: 29\n
        task = int(lines[task_start][:-1].split(": ")[-1])

        # Since the in-sample evaluation message follows directly after optimization is done, we use that to record
        # the number of generations. We account for the early stopping message if it did not run to 200 generations.
        ended_early = 0 if in_start - task_start == 201 else - 1
        generations_by_task[task] = in_start - (task_start + 1) - ended_early

        # Following the "INFO:root:Evaluating in sample:" message, symbolic default performance are printed
        # They are formatted as "INFO:root:[make_tuple(p, mkd)|0.8893]"
        # First is any number of best solutions from the pareto front. The last four are benchmark solutions.
        # It is possible that two equally good solutions are printed (i.e. same length and performance).
        expr_in_task = set()
        max_length = 0

        for in_sample_evaluation in lines[in_start + 1: out_start]:
            expr, length, score = parse_evaluation_line(in_sample_evaluation)
            # Pareto fronts may contain literal duplicates, so we filter those out manually.
            # We also do not want to include the baseline solutions (they have ':' in their line)
            if expr not in expr_in_task: # and ':' not in expr:
                expressions_by_length[length].append(expr)
                expr_in_task.add(expr)

            if length != 0:
                if length < 4:
                    # Only report one out-of-sample solution for each length (and all benchmarks), so overwrite is OK.
                    df.loc[task, "in-sample"][f"length-{length}"] = score

                # Update best so far score and maximum length
                df.loc[task, "in-sample"][f"final"] = np.nanmax(
                    [score, df.loc[task, "in-sample"][f"final"]])
                max_length = max(max_length, length)
            else:
                df.loc[task, "in-sample"][expr] = score

            if length > max_length:
                max_length = length  # To know for which length "best" should score out of sample

        # Because two equal solutions can be in the Pareto front,
        # we note the average out of sample performance if multiple solutions were found.
        # Naturally, the solutions with the best in-sample score were those with the highest length in the Pareto front.

        scores_by_length = defaultdict(list)

        for out_sample_evaluation in lines[out_start + 1: next_task]:
            expr, length, score = parse_evaluation_line(out_sample_evaluation)
            if length != 0:
                scores_by_length[length].append(score)
            else:
                df.loc[task, "out-sample"][expr] = score

        for length, scores in scores_by_length.items():
            if length < 4:
                df.loc[task, "out-sample"][f"length-{length}"] = np.mean(scores)
            if length == max_length:
                df.loc[task, "out-sample"][f"final"] = np.mean(scores)
            if np.mean(scores) == float("nan"):
                print('hi')

    return df, expressions_by_length, generations_by_task


def comparisons(df, sample="out-sample"):
    out_sample = df.index.map(lambda idx: idx[1] == sample)

    alone = {k: 0 for k in df.iloc[0].index.values}
    shared = {k: 0 for k in df.iloc[0].index.values}

    for _, out in df.loc[out_sample].iterrows():
        best = out[out == out.max()].index.values
        if len(best) == 1:
            alone[best[0]] += 1
        else:
            for winner in best:
                shared[winner] += 1

    alone = {k: alone[k] for k in sorted(alone)}
    shared = {k: shared[k] for k in sorted(shared)}
    either = {k: shared[k] + alone[k] for k in sorted({**alone, **shared})}
    comparison = pd.DataFrame([alone, shared, either], index=['alone', 'shared', 'either'])

    df_out = df.loc[out_sample].copy()
    df_out['max'] = df_out.max(axis=1)
    for col in df_out:
        df_out['d_' + col] = df_out['max'] - df_out[col]
    d_cols = [c for c in df_out.columns if c.startswith('d_')]
    df_out[d_cols].mean()
    df_out[d_cols].median()

    in_sample = df.index.map(lambda idx: idx[1] == "in-sample")
    df.loc[in_sample].idxmax(axis=1).value_counts()
    df.loc[in_sample][reversed(df.columns)].idxmax(axis=1).value_counts()
    return comparison, df_out


In [61]:
from collections import defaultdict
from typing import Tuple
import numpy as np
import pandas as pd

# Symbolic Defaults by 'complexity' of expression
In this notebook we take a look at the results of running the script at its default settings, this means:
 - evaluation across all tasks
 - recording the pareto front of symbolic defaults after each search
 - evaluating in-sample and out-of-sample performance of those dynamic defaults, as well as some pre-defined ones
 
**note:** The console cut off results for the first few tasks, so I am rerunning those now.

For each task we will extract:
 - the number of generations optimization ran for (max=200)
 - max length expression
 - in and out of sample performance for length 1, 2 and 3 expression

For each task save the benchmark results. We also save results for length 1, 2 and 3 solutions as well as the best one found (that may be longer). Specifically we record:
 - best in_sample performance at length 1, 2, 3
 - best in_sample performance for any length
 - average out_sample performance by length for length 1, 2, 3
 - average out_sample performance for the longest (i.e. best in-sample score) solution(s).

## Setup
We have experiment data for a set of algorithms and meta-data for the datasets on which the experiments took place.
We use symbolic regression to find an expression for symbolic default values that give good performance across tasks.
Symbolic regression is performed with leave-one-task-out, which means for each algorithm we have multiple searches for a symbolic default, and their performance is recorded for both in-sample (the optimization surface of all-but-one tasks) and out-of-sample (the left out task) performance. Performance here is solely based on surrogate model predictions, no additional experiments have been performed (yet).

In our search, we use NSGA-II selection to perform multi-objective optimization: find the expression with the best performance, while using the fewest number of operators (e.g. `divide`, `multiply`).

### Terms
**Length** of an expression denotes the number of operators in it. A symbolic value is *not* considered an operation.
Consider the following SVM defaults for cost and gamma:
 - `make_tuple`(m, mkd) is length 1.
 - `make_tuple`(m, `truediv`(mkd, xvar)) is length 2.
 - `make_tuple`(16., `truediv`(mkd, xvar)) is length 2.

The **final** solution refers to the symbolic default with the highest in-sample score for a task (regardless of its length). This means for each task there is *at least* one final solution, but there may be more and they are not of a specific length.


The **baseline** solutions are typically the default hyperparameter settings of mlr, scikit-learn, or both.

---

We read all the logs, because some logs are incomplete we have to explicitly give the name of the baselines (this will be fixed for future runs).

In [65]:
import os
baselines = dict(
    glmnet=["mlr_default", "sklearn_default"],
    kerasff=["initial_values"],
    knn=["mlr_default"],
    rf=["mlr_default"],
    rpart=["mlr_default"],
    asvm=["sklearn_scale", "symbolic_best", "symbolic_v2" , "const"],
)
traces = {}
for file in os.listdir('runs'):
    if file.endswith('.log'):
        baseline = []
        for method, bls in baselines.items():
            if method in file:
                baseline = bls
        traces[file[:-4]] = Trace(os.path.join('runs', file), benchmarks=baseline)

The predefined defaults are:
 * mlr_default := make_tuple(1., 0.01)
 * sklearn_default := make_tuple(1., 1.)
The predefined defaults are:
The predefined defaults are:
The predefined defaults are:
 * mlr_default := make_tuple(1., 16., 10., 200.)
The predefined defaults are:
The predefined defaults are:
 * mlr_default := make_tuple(0.01, 30., 1., 20.)
The predefined defaults are:
 * sklearn_scale := make_tuple(1., truediv(1., mul(p, xvar)))
 * symbolic_best := make_tuple(16., truediv(mkd, xvar))
 * symbolic_v2 := make_tuple(m, add(mkd, mkd)))
 * const := make_tuple(812.267350, 0.001361)
The predefined defaults are:
 * sklearn_scale := make_tuple(1., truediv(1., mul(p, xvar)))
 * symbolic_best := make_tuple(16., truediv(mkd, xvar))
 * symbolic_v2 := make_tuple(m, add(mkd, mkd)))
 * const := make_tuple(812.267350, 0.001361)


### Optimization
As described before, for each problem we find a symbolic default leaving one task out.
We are interested to see how fast the symbolic regression converges across tasks.

In [34]:
print("Median number of generations across tasks by problem:")
for log, trace in traces.items():
    print(f"{log: <15} {trace.generations_by_task.median().astype(int):3d}")

Median number of generations across tasks by problem:
svm_warm         46


In [35]:
import matplotlib.pyplot as plt
import math

fig, axes = plt.subplots(math.ceil(len(traces) / 4), 4, sharey=True, figsize=(16, 9))
for ax, (log, trace) in zip(axes.flatten(), traces.items()):
    traces[log].generations_by_task.hist(bins=20, ax=ax)
    ax.set_title(f"{log} ({len(trace.generations_by_task)} tasks)")
    ax.set_ylabel('count')
    ax.set_xlabel('generations')
fig.tight_layout()

The plot above shows a histogram counting the number of generations until stopping. These results were obtained with default setting of early stopping if no improvement was made after 20 generations, with a 200 generation maximum.

### Comparing Expressions
For a given problem, we have a Pareto front of solutions for search (=each left out task).
This Pareto front may contain "twins", multiple solutions which performance equally well and have the same length.
Given that the response surface does not differ *that* much when leaving any particular task out, we hope that the symbolic expressions we find are reasonably consistent across searches.
To have some indication of how consistent the results are, for each problem we find the most frequent solutions of length 1, 2 and 3. We also note the number of hyperparameters for which we aim to find a symbolic default, as we expect this to be correlated to how consistent the solutions are:

In [41]:
expr_count = pd.DataFrame(np.zeros((4, len(traces))), columns=list(traces), index=[1, 2, 3, "params"])
for log, trace in traces.items():  
    for length, expressions in sorted(trace.expressions.items()):
        if 0 < length < 4:
            m = max(set(expressions), key=expressions.count)
            expr_count.loc[length][log] = expressions.count(m)
            # print(f" Found {len(expressions):3d} expressions of length {length}. Most frequent: {m} ({expressions.count(m)} times)")
            if length == 1:
                expr_count.loc["params"][log] = m.count(',') + 1
expr_count

,svm_warm
1,106.0
2,85.0
3,24.0
params,2.0


We can look at the found expressions per problem:

In [37]:
for length, expressions in sorted(traces["svm_warm"].expressions.items()):
    if 0 < length < 4:
        m = max(set(expressions), key=expressions.count)
        expr_count.loc[length][log] = expressions.count(m)
        print(f" Found {len(expressions):3d} expressions of length {length}. Most frequent: {m} ({expressions.count(m)} times)")

 Found 106 expressions of length 1. Most frequent: make_tuple(p, mkd) (106 times)
 Found 106 expressions of length 2. Most frequent: make_tuple(m, truediv(mkd, xvar)) (85 times)
 Found  91 expressions of length 3. Most frequent: make_tuple(7.0, truediv(1.0, mul(p, xvar))) (24 times)


## Expression Quality
The expressions we find also need to be good.
Here we compare the following 'strategies':
 - length-*n*: always pick the best expression of length *n*
 - *final*: always pick the best expression, regardless of length
 - *baseline(s)*: compare it to baselines we defined
 
We want to know (all based on out-of-sample performance):
 - which strategy gives the best solution most often?
 - which strategy experiences the least mean regret?
 - which strategy experiences the least median regret?
 
As mentioned before, there can be "twins" in the Pareto front, which means multiple solutions with equal length have equal in-sample performance.
In this case we average the out-of-sample score of those twins.

#### number of wins:

The following table records the number of times a strategy led to the symbolic expression with the best out-of-sample performance (multiple strategies can be the best each task):

In [38]:
comparisons = pd.DataFrame()
for log, trace in traces.items():
    comparisons = comparisons.append(trace.comparison.loc['either'].rename(log))
comparisons

,const,final,length-1,length-2,length-3,sklearn_scale,symbolic_best,symbolic_v2
svm_warm,14.0,11.0,32.0,12.0,9.0,17.0,26.0,18.0


#### median regret:
The following table records the median regret for a specific strategy compared to picking the best in hindsight:

In [39]:
medians = pd.DataFrame()
for log, trace in traces.items():
    m = trace.d_scores.median().rename(log)
    medians = medians.append(m)
medians[[c for c in medians.columns if 'd_' in c and c != 'd_max']]

,d_const,d_final,d_length-1,d_length-2,d_length-3,d_sklearn_scale,d_symbolic_best,d_symbolic_v2
svm_warm,0.02645,0.0118,0.01285,0.0115,0.0117,0.01655,0.00525,0.01125


#### mean regret:
The following table records the mean regret for a specific strategy compared to picking the best in hindsight:

In [40]:
means = pd.DataFrame([])
for log, trace in traces.items():
    m = trace.d_scores.mean().rename(log)
    means = means.append(m)
means[[c for c in medians.columns if 'd_' in c and c != 'd_max']]

,d_const,d_final,d_length-1,d_length-2,d_length-3,d_sklearn_scale,d_symbolic_best,d_symbolic_v2
svm_warm,0.071775,0.039659,0.043031,0.035999,0.037292,0.046048,0.037642,0.047946


### Sanity Checks
Sometimes out-of-sample performance of a baseline may still be better than that of our solution.
However, in-sample performance of our own solutions should always be better than any baseline.
If that is not the case, this would indicate our search does not explore the space well enough.

In [55]:
comparisons = pd.DataFrame()
for log, trace in traces.items():
    comparisons = comparisons.append(trace.in_comparison.loc['either'].rename(log))
comparisons

,const,final,length-1,length-2,length-3,sklearn_scale,symbolic_best,symbolic_v2
svm_warm,0.0,106.0,0.0,23.0,43.0,0.0,0.0,0.0


----
End of notebook - just sketchpad below